# Training the models
In this notebook we will be training several different and increasingly more complex Deep Q-Learning models and saving their best result and run logs alongside. We will be experimenting with different types of greedy policies and improvements pon the deep learning model, such as: dueling networks and double deep q learning.

**NOTE ON RUNNING WITH TENSORFLOW** The infrequent updates actually make the GPU backend slower than CPU due to data transfer overhead. If using tensorflow as a backend for keras, it is best to use the CPU backend with optimizations.

In [1]:
import sys
import os

from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent, Sequential
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy, BoltzmannQPolicy
from rl.callbacks import FileLogger

# add local path
sys.path.append(os.path.join(os.getcwd(), "./src"))

from rl_model import TorcsKerasTrainer

/home/nikitautiu/envs/mirpr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
NB_OF_RUNS = 10  # the number of runs for each one of experiments
NB_STEPS = 100000  # the number of steps for each episode 

## Simple deep q-learning
We begin with defining the model for our most simple method. This is a simple deep q-learning model with an Epsilon Q-greedy policy. The epsilon is fixed at 5%. The deep model is a 3-layer 35-25-18 network with relu activations. 10 training of 10^5 steps each are performed,.

In [3]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=EpsGreedyQPolicy(0.05))
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

Run the simulation for the given number of times(10).

In [ ]:
NB_OF_RUNS = 10

for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/simple_dqn_best_{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/simple_dqn_{}.json'.format(run_nb))])

## Linearly annealed eps
Thi one is the same as the one before, but the epsilon starts large and slowly decreases over time. In theory this should increase exploration.

In [5]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.05,
                                  value_test=.0, nb_steps=40000)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=policy)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    
    return dqn

In [ ]:
for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/simple_dqn_linear_eps_best_{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/simple_dqn_linear_eps_{}.json'.format(run_nb))], min_step_save=40000)

## Boltzmann Exploration

In [7]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)
    policy = BoltzmannQPolicy()

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=policy)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    
    return dqn

In [ ]:
for run_nb in range(NB_OF_RUNS):
    yqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/simple_dqn_boltzmann_best_{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/simple_dqn_boltzmann_{}.json'.format(run_nb))])

## Dueling DQN
Uses two networks, one predicting the reward for a particular action, and one for the state. The results are then averaged.

In [9]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=EpsGreedyQPolicy(0.05), enable_dueling_network=True,
                   dueling_type='avg')
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

In [ ]:
for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/dueling_dqn_best-{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/dueling_dqn_{}.json'.format(run_nb))])

Waiting for server on 3101............
Count Down : 5
Waiting for server on 3101............
Count Down : 4
Waiting for server on 3101............
Count Down : 3
Waiting for server on 3101............
Count Down : 2
Waiting for server on 3101............
Count Down : 1
Waiting for server on 3101............
Count Down : 0
Waiting for server on 3101............
Count Down : -1
relaunch torcs
Waiting for server on 3101............
Count Down : 4
Waiting for server on 3101............
Count Down : 3
Waiting for server on 3101............
Count Down : 2
Waiting for server on 3101............
Count Down : 1
Waiting for server on 3101............
Count Down : 0
Waiting for server on 3101............
Count Down : -1
relaunch torcs
Client connected on 3101..............
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_30 (Flatten)         (None, 29)                0         
_____________________________

 3021/10000 [========>.....................] - ETA: 2:01 - reward: 25.7573Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3066/10000 [========>.....................] - ETA: 2:01 - reward: 25.4345Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3130/10000 [========>.....................] - ETA: 2:01 - reward: 25.2888Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3246/10000 [========>.....................] - ETA: 1:58 - reward: 25.1575Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3313/10000 [========>.....................] - ETA: 1:57 - reward: 25.0526Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3380/10000 [=========>....................] - ETA: 1:57 - reward: 24.9151Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 6830/10000 [===================>..........] - ETA: 57s - reward: 23.9071Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6923/10000 [===================>..........] - ETA: 55s - reward: 23.9484Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7011/10000 [====================>.........] - ETA: 54s - reward: 23.9401Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7106/10000 [====================>.........] - ETA: 52s - reward: 24.1136Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7202/10000 [====================>.........] - ETA: 50s - reward: 24.2648Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7300/10000 [====================>.........] - ETA: 48s - reward: 24.2822Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 735

 1095/10000 [==>...........................] - ETA: 3:37 - reward: 20.2193Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1130/10000 [==>...........................] - ETA: 3:41 - reward: 20.1406Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1169/10000 [==>...........................] - ETA: 3:43 - reward: 19.6916Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1204/10000 [==>...........................] - ETA: 3:47 - reward: 19.7717Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1246/10000 [==>...........................] - ETA: 3:50 - reward: 20.4211Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 1281/10000 [==>...........................] - ETA: 3:52 - reward: 20.4068Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 3816/10000 [==========>...................] - ETA: 2:46 - reward: 43.2029Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3869/10000 [==========>...................] - ETA: 2:45 - reward: 43.3560Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3924/10000 [==========>...................] - ETA: 2:44 - reward: 43.5340Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 3996/10000 [==========>...................] - ETA: 2:42 - reward: 44.0125Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4048/10000 [===========>..................] - ETA: 2:41 - reward: 44.1879Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4097/10000 [===========>..................] - ETA: 2:40 - reward: 44.3076Waiting for server on 3101............
Count Down : 5
Client connected on 3101.............

 7130/10000 [====================>.........] - ETA: 1:16 - reward: 43.8655Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7632/10000 [=====================>........] - ETA: 1:00 - reward: 42.0039Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7684/10000 [======================>.......] - ETA: 59s - reward: 42.0919Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7973/10000 [======================>.......] - ETA: 51s - reward: 41.6306
Reward of 8486.261653403255 for episode 232 better than last -- SAVING

Client connected on 3101..............
 8334/10000 [========================>.....] - ETA: 41s - reward: 39.7478Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 8388/10000 [========================>.....] - ETA: 40s - reward: 39.5453Waiting for server on 3101............
Count Down : 5
Client connected on 31

 4120/10000 [===========>..................] - ETA: 1:09 - reward: 8.3219Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 4622/10000 [============>.................] - ETA: 1:04 - reward: 8.8001Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5124/10000 [==============>...............] - ETA: 58s - reward: 7.8535Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 5647/10000 [===============>..............] - ETA: 52s - reward: 7.6130Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6149/10000 [=================>............] - ETA: 47s - reward: 6.9738Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6848/10000 [===================>..........] - ETA: 38s - reward: 7.4200Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7350/10

 8331/10000 [=======================>......] - ETA: 20s - reward: 91.9133Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 8927/10000 [=========================>....] - ETA: 13s - reward: 92.8970Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9828/10000 [============================>.] - ETA: 2s - reward: 93.1052Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
10000/10000 [==============================] - 125s 13ms/step - reward: 93.3534
17 episodes - episode_reward: 56978.624 [14591.941, 146135.039] - loss: 47156.078 - mean_absolute_error: 5094.264 - mean_q: 5509.278

Interval 8 (70000 steps performed)
   42/10000 [..............................] - ETA: 2:26 - reward: 88.3885 Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
  152/10000 [..............................] - ETA: 3:17 - reward: 80.3295Waiting for 

 6097/10000 [=================>............] - ETA: 54s - reward: 90.6162Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6279/10000 [=================>............] - ETA: 52s - reward: 90.5998Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6530/10000 [==================>...........] - ETA: 48s - reward: 90.6474Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6761/10000 [===================>..........] - ETA: 45s - reward: 90.8709Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 6940/10000 [===================>..........] - ETA: 43s - reward: 90.8156Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 7035/10000 [====================>.........] - ETA: 42s - reward: 90.7929Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 713

 8990/10000 [=========================>....] - ETA: 14s - reward: 100.8542Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9060/10000 [==========================>...] - ETA: 13s - reward: 100.5564Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9333/10000 [==========================>...] - ETA: 9s - reward: 100.7512Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
 9539/10000 [===========================>..] - ETA: 6s - reward: 100.3918Waiting for server on 3101............
Count Down : 5
Client connected on 3101..............
10000/10000 [==============================] - 142s 14ms/step - reward: 100.8531
done, took 1469.173 seconds
Waiting for server on 3101............
Count Down : 5
Waiting for server on 3101............
Count Down : 4
Waiting for server on 3101............
Count Down : 3
Waiting for server on 3101............
Count Down : 2
Waiting f

## Double DQN
This approach separates the model network from the q-value network. Basically a network predicts the actions(like a policy function) and the other one predicts the q value of the given action.

In [ ]:
def build_model(obs_shape, nb_actions):
    # Next, we build a very simple model.
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + obs_shape))
    model.add(Dense(35))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
    # even the metrics!
    memory = SequentialMemory(limit=NB_STEPS, window_length=1)

    # deep q learning agent, uses greedy-epsilon policy, does NB_STEPS steps of warmup and
    # updates the gradient every 1000 steps
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
                   target_model_update=1e-2, policy=EpsGreedyQPolicy(0.05), enable_double_dqn=True)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

In [ ]:
for run_nb in range(NB_OF_RUNS):
    dqn = TorcsKerasTrainer(throttle=True, discrete_actions=9, model_function=build_model,
                            obs_fields=['speedX', 'speedY', 'speedZ', 'track', 'focus', 'trackPos', 'angle'])
    # saving best model fo reach on of the runs
    # laos log every episode result in a separate file for later use
    dqn.fit(nb_steps=NB_STEPS, best_filename='saved_weights/double_dqn_best-{}.h5f'.format(run_nb), verbose=1,
            callbacks=[FileLogger('logs/double_dqn_{}.json'.format(run_nb))])